In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

NameError: name 'plt' is not defined

In [2]:
# 1 Filtering
import findspark
findspark.init()

import pandas as pd

# read T2
df=pd.read_csv("prueba.csv")

# list the columns
list(df)

# print number of rows and columns 
print (df.shape)
df.head()


FileNotFoundError: [Errno 2] File b'prueba.csv' does not exist: b'prueba.csv'

In [ ]:
#Catch training and test values from the dataset
from sklearn.model_selection import train_test_split
import numpy as np

from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB


#featuredColumns = ['LinearAcceleration_x_MEAN']
#featuredColumns = ['Pressure_MEAN']
#featuredColumns = ['MagneticField_COV_z_x']

#HERE START NAIVE BAYES LEARNER
#rng = np.random.RandomState(2)
#X += 2 * rng.uniform(size=X.shape)

# scale
#X = MinMaxScaler().fit_transform(X)


featuredColumns = ['sample','PP','PSOE','Cs','PODEMOS','PPresultado','PSOEresultado','Csresultado','VOXresultado','PODEMOSresultado']

X = df[featuredColumns]
Y = df['PSOEresultado']
Y_round =  Y.round(0) 
X_scaled=StandardScaler().fit(X)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y_round,test_size=0.2)

In [ ]:
#Define the algorithm to use
from sklearn.naive_bayes import GaussianNB

algoritm=GaussianNB()

#Train the model
algoritm.fit(X_train,Y_train)

#Prediction
Y_pred_naive_bayes = algoritm.predict(X_test)


In [ ]:
from sklearn.metrics import mean_absolute_error
mea = mean_absolute_error(Y_test, Y_pred_naive_bayes)
#print "Error Measure ",mae

# PLOTTING
import numpy as np
xx =np.stack(i for i in range(len(Y_test)))
plt.scatter(xx,Y_test,c='r',label='data')
plt.plot(xx,Y_pred_naive_bayes,c='g',label='prediction')
plt.axis('tight')
plt.legend()
plt.title("GaussianNaiveBayes")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

#Confusion matrix
matrix = confusion_matrix(Y_test,Y_pred_naive_bayes)
print("Matriz de confusion:")
print(matrix)
plot_confusion_matrix(Y_test, Y_pred_naive_bayes , classes = unique_labels(Y_test, Y_pred_naive_bayes), normalize=False,
                      title='Normalized confusion matrix')

In [ ]:
from sklearn.preprocessing import StandardScaler

#Normalize the data
escaler=StandardScaler()
X_train=escaler.fit_transform(X_train)
X_test=escaler.fit_transform(X_test)

In [ ]:
#Define the algorithm to use
from sklearn.linear_model import LogisticRegression

algoritm=LogisticRegression(solver='lbfgs')

In [ ]:
#Train the model
algoritm.fit(X_train,Y_train.values.ravel())

In [ ]:
#Prediction
Y_pred_naive_bayes = algoritm.predict(X_test)

In [ ]:
#Precision of the algorithm
from sklearn.metrics import precision_score 

precision = precision_score(Y_test, Y_pred_naive_bayes, average='weighted', labels= pd.unique(Y_pred_naive_bayes))
print("Precision del modelo:")
print(precision)

In [ ]:
#HERE START DECISION TREE LEARNER
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

sqlCtx = SQLContext(sc)

#Convert the dataframe from pandas to sql
dfMondays = sqlCtx.createDataFrame(dfMondays)

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#Merge selected columns into one
assembler = VectorAssembler(inputCols = featuredColumns, outputCol="features")
assembled = assembler.transform(dfMondays)

#Catch train and test data from chosen columns
(trainingData, testData) = assembled.randomSplit([0.8,0.2], seed=123123) 

trainingData.count(), testData.count()

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

#Start decision tree clasificator 
dt = DecisionTreeClassifier(labelCol="attack", featuresCol="features", maxDepth=5, minInstancesPerNode=20, impurity="gini")

In [ ]:
from pyspark.ml import Pipeline

#Normalize data
pipeline = Pipeline(stages=[dt])
model = pipeline.fit(trainingData)

In [ ]:
#Get predictions from the model

predictions = model.transform(testData)

In [ ]:
from pyspark.sql.types import DoubleType

#Convert the attack column from int to double to prevent subsequent failure
predictions = predictions.withColumn("attack", predictions["attack"].cast("double"))

In [ ]:
#Compare first 10 rows (not necesary)
predictions = predictions.select("prediction","attack")
predictions.show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#Show success percentage of the model with the test data
evaluator = MulticlassClassificationEvaluator(labelCol="attack", predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print("Accuracy:")
print(accuracy)

In [ ]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(predictions.rdd.map(tuple))

In [ ]:
#Confusion matrix from the decision tree
metrics.confusionMatrix().toArray().transpose()

In [ ]:
plot_confusion_matrix(Y_test, Y_pred_naive_bayes , classes = unique_labels(Y_test, Y_pred_naive_bayes), normalize=True,
                      title='Normalized confusion matrix')